## Final project Coursera Data Science Capstone


In [1]:
import numpy as np
import folium
import pandas as pd 
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [2]:
CLIENT_ID = 'QU44LRRNNTD1LK5NSK5RWM3J52HINNK1LOBWYL1OUXUZR00G' # your Foursquare ID
CLIENT_SECRET = '500HBAWIOPLDGIQJOSQZRTR42H54RDY0TKQ33MYSYWC14YF5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QU44LRRNNTD1LK5NSK5RWM3J52HINNK1LOBWYL1OUXUZR00G
CLIENT_SECRET:500HBAWIOPLDGIQJOSQZRTR42H54RDY0TKQ33MYSYWC14YF5


In [3]:
latitude = 21.1236
longitude = -101.6821
print(latitude, longitude)

21.1236 -101.6821


In [4]:
search_query = 'Walmart'
radius = 50000
LIMIT = 30

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [6]:
results = requests.get(url).json()
#results

In [7]:
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(30)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,4bbe2a449474c9b6712ed9b6,Blvd. Manuel J. Clouthier 302,MX,León,México,Bosques del Campestre,4320,[Blvd. Manuel J. Clouthier 302 (Bosques del Ca...,"[{'label': 'display', 'lat': 21.16149885883102...",21.161499,-101.691081,37125,Guanajuato,Walmart,v-1605658232
1,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,4be5d857d4f7c9b648052620,Blvd. Juan José Torres Landa 1601,MX,León,México,NaN,3601,"[Blvd. Juan José Torres Landa 1601, 37178 León...","[{'label': 'display', 'lat': 21.09832200719789...",21.098322,-101.660449,37178,Guanajuato,Walmart,v-1605658232
2,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,4c09c20a340720a1b4708593,Galerias Las Torres,MX,León,México,Blvd. Juan Alonso de Torres 1325,4503,[Galerias Las Torres (Blvd. Juan Alonso de Tor...,"[{'label': 'display', 'lat': 21.14700345173944...",21.147003,-101.646720,37178,Guanajuato,Walmart,v-1605658232
3,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,50358829e4b0aab52bd7e343,Blvd. La Luz,MX,León,México,Blvd. Delta,7653,"[Blvd. La Luz (Blvd. Delta), 3720 León, Guanaj...","[{'label': 'display', 'lat': 21.11007609214700...",21.110076,-101.609840,3720,Guanajuato,Walmart,v-1605658232
4,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,5dbb9a306dfe8f00086dd670,NaN,MX,León,México,NaN,9476,"[37295 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.079158, 'lng':...",21.079158,-101.604270,37295,Guanajuato,Walmart Via Alta,v-1605658232
5,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,54a181e8498e3242ba86a427,Blvd. Morelos,MX,León,México,NaN,7576,"[Blvd. Morelos, 37665 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.17264957202568...",21.172650,-101.732696,37665,Guanajuato,Walmart La Antorcha,v-1605658232
6,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,54a186dd498e8fb585d6732f,Blvd. Francisco Villa N° 1402,MX,León,México,NaN,3132,"[Blvd. Francisco Villa N° 1402, 37278 León, Gu...","[{'label': 'display', 'lat': 21.12962264302637...",21.129623,-101.652633,37278,Guanajuato,Walmart,v-1605658232
7,"[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",False,52fa585a11d29516fb7ebf3e,Fco Villa,MX,NaN,México,Blvd Torres Landa,3544,"[Fco Villa (Blvd Torres Landa), México]","[{'label': 'display', 'lat': 21.098757, 'lng':...",21.098757,-101.660747,NaN,NaN,At&t (walmart),v-1605658232
8,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,5494e47a498ecffa71a10e3e,Boulevard Francisco Villa 1402,MX,Villa de laa flores,México,Fray Daniel Mireles,3578,[Boulevard Francisco Villa 1402 (Fray Daniel M...,"[{'label': 'display', 'lat': 21.12338974610575...",21.123390,-101.647636,NaN,Guanajuato,Walmart Francisco Villa,v-1605658232
9,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,5bf1d8c182a750002c071844,NaN,MX,León,México,NaN,7816,"[37179 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.132607, 'lng':...",21.132607,-101.607440,37179,Guanajuato,Walmart Punta Este,v-1605658232


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Walmart,Big Box Store,Blvd. Manuel J. Clouthier 302,MX,León,México,Bosques del Campestre,4320,[Blvd. Manuel J. Clouthier 302 (Bosques del Ca...,"[{'label': 'display', 'lat': 21.16149885883102...",21.161499,-101.691081,37125,Guanajuato,4bbe2a449474c9b6712ed9b6
1,Walmart,Big Box Store,Blvd. Juan José Torres Landa 1601,MX,León,México,NaN,3601,"[Blvd. Juan José Torres Landa 1601, 37178 León...","[{'label': 'display', 'lat': 21.09832200719789...",21.098322,-101.660449,37178,Guanajuato,4be5d857d4f7c9b648052620
2,Walmart,Big Box Store,Galerias Las Torres,MX,León,México,Blvd. Juan Alonso de Torres 1325,4503,[Galerias Las Torres (Blvd. Juan Alonso de Tor...,"[{'label': 'display', 'lat': 21.14700345173944...",21.147003,-101.646720,37178,Guanajuato,4c09c20a340720a1b4708593
3,Walmart,Big Box Store,Blvd. La Luz,MX,León,México,Blvd. Delta,7653,"[Blvd. La Luz (Blvd. Delta), 3720 León, Guanaj...","[{'label': 'display', 'lat': 21.11007609214700...",21.110076,-101.609840,3720,Guanajuato,50358829e4b0aab52bd7e343
4,Walmart Via Alta,Big Box Store,NaN,MX,León,México,NaN,9476,"[37295 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.079158, 'lng':...",21.079158,-101.604270,37295,Guanajuato,5dbb9a306dfe8f00086dd670
5,Walmart La Antorcha,Supermarket,Blvd. Morelos,MX,León,México,NaN,7576,"[Blvd. Morelos, 37665 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.17264957202568...",21.172650,-101.732696,37665,Guanajuato,54a181e8498e3242ba86a427
6,Walmart,Big Box Store,Blvd. Francisco Villa N° 1402,MX,León,México,NaN,3132,"[Blvd. Francisco Villa N° 1402, 37278 León, Gu...","[{'label': 'display', 'lat': 21.12962264302637...",21.129623,-101.652633,37278,Guanajuato,54a186dd498e8fb585d6732f
7,At&t (walmart),Distribution Center,Fco Villa,MX,NaN,México,Blvd Torres Landa,3544,"[Fco Villa (Blvd Torres Landa), México]","[{'label': 'display', 'lat': 21.098757, 'lng':...",21.098757,-101.660747,NaN,NaN,52fa585a11d29516fb7ebf3e
8,Walmart Francisco Villa,Big Box Store,Boulevard Francisco Villa 1402,MX,Villa de laa flores,México,Fray Daniel Mireles,3578,[Boulevard Francisco Villa 1402 (Fray Daniel M...,"[{'label': 'display', 'lat': 21.12338974610575...",21.123390,-101.647636,NaN,Guanajuato,5494e47a498ecffa71a10e3e
9,Walmart Punta Este,Supermarket,NaN,MX,León,México,NaN,7816,"[37179 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.132607, 'lng':...",21.132607,-101.607440,37179,Guanajuato,5bf1d8c182a750002c071844


In [9]:
dataframe_filtered['categories'] = dataframe_filtered['categories'].astype(str)

In [10]:
dataframe_filtered.dtypes
possible = ['Supermarket', 'Big Box Store']

In [11]:
dataframe_filtered = dataframe_filtered[dataframe_filtered.categories.isin(possible)]

In [12]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Walmart,Big Box Store,Blvd. Manuel J. Clouthier 302,MX,León,México,Bosques del Campestre,4320,[Blvd. Manuel J. Clouthier 302 (Bosques del Ca...,"[{'label': 'display', 'lat': 21.16149885883102...",21.161499,-101.691081,37125,Guanajuato,4bbe2a449474c9b6712ed9b6
1,Walmart,Big Box Store,Blvd. Juan José Torres Landa 1601,MX,León,México,NaN,3601,"[Blvd. Juan José Torres Landa 1601, 37178 León...","[{'label': 'display', 'lat': 21.09832200719789...",21.098322,-101.660449,37178,Guanajuato,4be5d857d4f7c9b648052620
2,Walmart,Big Box Store,Galerias Las Torres,MX,León,México,Blvd. Juan Alonso de Torres 1325,4503,[Galerias Las Torres (Blvd. Juan Alonso de Tor...,"[{'label': 'display', 'lat': 21.14700345173944...",21.147003,-101.646720,37178,Guanajuato,4c09c20a340720a1b4708593
3,Walmart,Big Box Store,Blvd. La Luz,MX,León,México,Blvd. Delta,7653,"[Blvd. La Luz (Blvd. Delta), 3720 León, Guanaj...","[{'label': 'display', 'lat': 21.11007609214700...",21.110076,-101.609840,3720,Guanajuato,50358829e4b0aab52bd7e343
4,Walmart Via Alta,Big Box Store,NaN,MX,León,México,NaN,9476,"[37295 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.079158, 'lng':...",21.079158,-101.604270,37295,Guanajuato,5dbb9a306dfe8f00086dd670
5,Walmart La Antorcha,Supermarket,Blvd. Morelos,MX,León,México,NaN,7576,"[Blvd. Morelos, 37665 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.17264957202568...",21.172650,-101.732696,37665,Guanajuato,54a181e8498e3242ba86a427
6,Walmart,Big Box Store,Blvd. Francisco Villa N° 1402,MX,León,México,NaN,3132,"[Blvd. Francisco Villa N° 1402, 37278 León, Gu...","[{'label': 'display', 'lat': 21.12962264302637...",21.129623,-101.652633,37278,Guanajuato,54a186dd498e8fb585d6732f
8,Walmart Francisco Villa,Big Box Store,Boulevard Francisco Villa 1402,MX,Villa de laa flores,México,Fray Daniel Mireles,3578,[Boulevard Francisco Villa 1402 (Fray Daniel M...,"[{'label': 'display', 'lat': 21.12338974610575...",21.123390,-101.647636,NaN,Guanajuato,5494e47a498ecffa71a10e3e
9,Walmart Punta Este,Supermarket,NaN,MX,León,México,NaN,7816,"[37179 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.132607, 'lng':...",21.132607,-101.607440,37179,Guanajuato,5bf1d8c182a750002c071844


In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [14]:
search_query_soriana = 'Soriana'
radius = 50000
LIMIT = 30

In [15]:
url_soriana = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_soriana, radius, LIMIT)

In [16]:
results_soriana = requests.get(url_soriana).json()

In [17]:
venues_soriana = results_soriana['response']['venues']
# tranform venues into a dataframe
dataframe_soriana = json_normalize(venues_soriana)
dataframe_soriana.head(30)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4c17acaec2dfc9283f61a869,Malecon,MX,León,México,NaN,2893,"[Malecon, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.1067714166875,...",21.106771,-101.660859,NaN,Guanajuato,Soriana,v-1605658234
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4bbc89c3e45295218bfb55a4,Torres Landa,MX,León,México,Miguel de Cervantes,3728,"[Torres Landa (Miguel de Cervantes), 37000 Leó...","[{'label': 'display', 'lat': 21.10203753196693...",21.102038,-101.709580,37000,Guanajuato,Soriana,v-1605658234
2,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4d59f14cfe7c224b51c17db5,NaN,MX,León,México,NaN,5325,"[León, Guanajuato, México]","[{'label': 'display', 'lat': 21.14004985766762...",21.140050,-101.633941,NaN,Guanajuato,Soriana,v-1605658234
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4ded96e045dd3aa41d69d08e,Rio Mayo Real,MX,León,México,NaN,6189,"[Rio Mayo Real, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.08321104715064...",21.083211,-101.641143,NaN,Guanajuato,Soriana,v-1605658234
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5685d2af498e4403d45a5eaa,NaN,MX,NaN,México,NaN,3787,[México],"[{'label': 'display', 'lat': 21.10133556248182...",21.101336,-101.709679,NaN,NaN,Soriana,v-1605658234
5,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",False,5ecb88fe106cbe0008cb7966,Colonia El Tlacuache Pte. Blvd. Mariano Escobe...,MX,León,México,NaN,2957,[Colonia El Tlacuache Pte. Blvd. Mariano Escob...,"[{'label': 'display', 'lat': 21.1063842, 'lng'...",21.106384,-101.660404,37500,Guanajuato,Soriana Malecon Distribuidor Autorizado Telcel,v-1605658234
6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5658875b38faef99b03b2ea7,NaN,MX,NaN,México,NaN,4475,[México],"[{'label': 'display', 'lat': 21.15537282357526...",21.155373,-101.708506,NaN,NaN,Soriana Hiper,v-1605658234
7,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,547bd7de498e625c560e54a4,NaN,MX,NaN,México,NaN,4560,[México],"[{'label': 'display', 'lat': 21.15612372213005...",21.156124,-101.708801,NaN,NaN,Soriana Cerro Gordo,v-1605658234
8,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",False,54791df6498e43be5a5078b1,NaN,MX,NaN,México,NaN,4569,[México],"[{'label': 'display', 'lat': 21.15607538245781...",21.156075,-101.709012,NaN,NaN,soriana cerro gordo,v-1605658234
9,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4f7b86e8e4b0ec1aa389f09b,NaN,MX,NaN,México,NaN,2883,[México],"[{'label': 'display', 'lat': 21.106631, 'lng':...",21.106631,-101.661126,NaN,NaN,Zong Sheng Express (Soriana),v-1605658234


In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_soriana = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_soriana = dataframe_soriana.loc[:, filtered_columns_soriana]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_soriana['categories'] = dataframe_filtered_soriana.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_soriana.columns = [column.split('.')[-1] for column in dataframe_filtered_soriana.columns]

dataframe_filtered_soriana

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Soriana,Supermarket,Malecon,MX,León,México,NaN,2893,"[Malecon, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.1067714166875,...",21.106771,-101.660859,NaN,Guanajuato,4c17acaec2dfc9283f61a869
1,Soriana,Supermarket,Torres Landa,MX,León,México,Miguel de Cervantes,3728,"[Torres Landa (Miguel de Cervantes), 37000 Leó...","[{'label': 'display', 'lat': 21.10203753196693...",21.102038,-101.709580,37000,Guanajuato,4bbc89c3e45295218bfb55a4
2,Soriana,Supermarket,NaN,MX,León,México,NaN,5325,"[León, Guanajuato, México]","[{'label': 'display', 'lat': 21.14004985766762...",21.140050,-101.633941,NaN,Guanajuato,4d59f14cfe7c224b51c17db5
3,Soriana,Grocery Store,Rio Mayo Real,MX,León,México,NaN,6189,"[Rio Mayo Real, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.08321104715064...",21.083211,-101.641143,NaN,Guanajuato,4ded96e045dd3aa41d69d08e
4,Soriana,Grocery Store,NaN,MX,NaN,México,NaN,3787,[México],"[{'label': 'display', 'lat': 21.10133556248182...",21.101336,-101.709679,NaN,NaN,5685d2af498e4403d45a5eaa
5,Soriana Malecon Distribuidor Autorizado Telcel,Electronics Store,Colonia El Tlacuache Pte. Blvd. Mariano Escobe...,MX,León,México,NaN,2957,[Colonia El Tlacuache Pte. Blvd. Mariano Escob...,"[{'label': 'display', 'lat': 21.1063842, 'lng'...",21.106384,-101.660404,37500,Guanajuato,5ecb88fe106cbe0008cb7966
6,Soriana Hiper,Grocery Store,NaN,MX,NaN,México,NaN,4475,[México],"[{'label': 'display', 'lat': 21.15537282357526...",21.155373,-101.708506,NaN,NaN,5658875b38faef99b03b2ea7
7,Soriana Cerro Gordo,Convenience Store,NaN,MX,NaN,México,NaN,4560,[México],"[{'label': 'display', 'lat': 21.15612372213005...",21.156124,-101.708801,NaN,NaN,547bd7de498e625c560e54a4
8,soriana cerro gordo,Market,NaN,MX,NaN,México,NaN,4569,[México],"[{'label': 'display', 'lat': 21.15607538245781...",21.156075,-101.709012,NaN,NaN,54791df6498e43be5a5078b1
9,Zong Sheng Express (Soriana),Chinese Restaurant,NaN,MX,NaN,México,NaN,2883,[México],"[{'label': 'display', 'lat': 21.106631, 'lng':...",21.106631,-101.661126,NaN,NaN,4f7b86e8e4b0ec1aa389f09b


In [19]:
dataframe_filtered_soriana['categories'] = dataframe_filtered_soriana['categories'].astype(str)

In [20]:
possible_soriana = ['Supermarket', 'Market', 'Grocery Store']

In [21]:
dataframe_filtered_soriana = dataframe_filtered_soriana[dataframe_filtered_soriana.categories.isin(possible_soriana)]

In [22]:
dataframe_filtered_soriana

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Soriana,Supermarket,Malecon,MX,León,México,NaN,2893,"[Malecon, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.1067714166875,...",21.106771,-101.660859,NaN,Guanajuato,4c17acaec2dfc9283f61a869
1,Soriana,Supermarket,Torres Landa,MX,León,México,Miguel de Cervantes,3728,"[Torres Landa (Miguel de Cervantes), 37000 Leó...","[{'label': 'display', 'lat': 21.10203753196693...",21.102038,-101.709580,37000,Guanajuato,4bbc89c3e45295218bfb55a4
2,Soriana,Supermarket,NaN,MX,León,México,NaN,5325,"[León, Guanajuato, México]","[{'label': 'display', 'lat': 21.14004985766762...",21.140050,-101.633941,NaN,Guanajuato,4d59f14cfe7c224b51c17db5
3,Soriana,Grocery Store,Rio Mayo Real,MX,León,México,NaN,6189,"[Rio Mayo Real, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.08321104715064...",21.083211,-101.641143,NaN,Guanajuato,4ded96e045dd3aa41d69d08e
4,Soriana,Grocery Store,NaN,MX,NaN,México,NaN,3787,[México],"[{'label': 'display', 'lat': 21.10133556248182...",21.101336,-101.709679,NaN,NaN,5685d2af498e4403d45a5eaa
6,Soriana Hiper,Grocery Store,NaN,MX,NaN,México,NaN,4475,[México],"[{'label': 'display', 'lat': 21.15537282357526...",21.155373,-101.708506,NaN,NaN,5658875b38faef99b03b2ea7
8,soriana cerro gordo,Market,NaN,MX,NaN,México,NaN,4569,[México],"[{'label': 'display', 'lat': 21.15607538245781...",21.156075,-101.709012,NaN,NaN,54791df6498e43be5a5078b1
11,Soriana Hiper,Supermarket,Plaza La Joya,MX,Silao,México,5 de Mayo 112,33804,"[Plaza La Joya (5 de Mayo 112), 36100 Silao, G...","[{'label': 'display', 'lat': 20.94633894668505...",20.946339,-101.417925,36100,Guanajuato,4cfbe4effeec6dcbed3e4936
13,Soriana Express,Grocery Store,Blvd. Luis H. Ducoing 144,MX,Silao,México,NaN,32600,"[Blvd. Luis H. Ducoing 144, Silao, Guanajuato,...","[{'label': 'display', 'lat': 20.95489779939847...",20.954898,-101.425613,NaN,Guanajuato,4f5ebdf1e4b03d72021d34c6
15,Mercado Soriana,Grocery Store,NaN,MX,Lagos de Moreno,México,NaN,37651,"[Lagos de Moreno, Jalisco, México]","[{'label': 'display', 'lat': 21.36022220370403...",21.360222,-101.941400,NaN,Jalisco,4e3838351f6ebc502ce87c09


In [23]:
dataframe_filtered_soriana = dataframe_filtered_soriana.loc[0:8,:]

In [24]:
dataframe_filtered_soriana

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Soriana,Supermarket,Malecon,MX,León,México,NaN,2893,"[Malecon, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.1067714166875,...",21.106771,-101.660859,NaN,Guanajuato,4c17acaec2dfc9283f61a869
1,Soriana,Supermarket,Torres Landa,MX,León,México,Miguel de Cervantes,3728,"[Torres Landa (Miguel de Cervantes), 37000 Leó...","[{'label': 'display', 'lat': 21.10203753196693...",21.102038,-101.709580,37000,Guanajuato,4bbc89c3e45295218bfb55a4
2,Soriana,Supermarket,NaN,MX,León,México,NaN,5325,"[León, Guanajuato, México]","[{'label': 'display', 'lat': 21.14004985766762...",21.140050,-101.633941,NaN,Guanajuato,4d59f14cfe7c224b51c17db5
3,Soriana,Grocery Store,Rio Mayo Real,MX,León,México,NaN,6189,"[Rio Mayo Real, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.08321104715064...",21.083211,-101.641143,NaN,Guanajuato,4ded96e045dd3aa41d69d08e
4,Soriana,Grocery Store,NaN,MX,NaN,México,NaN,3787,[México],"[{'label': 'display', 'lat': 21.10133556248182...",21.101336,-101.709679,NaN,NaN,5685d2af498e4403d45a5eaa
6,Soriana Hiper,Grocery Store,NaN,MX,NaN,México,NaN,4475,[México],"[{'label': 'display', 'lat': 21.15537282357526...",21.155373,-101.708506,NaN,NaN,5658875b38faef99b03b2ea7
8,soriana cerro gordo,Market,NaN,MX,NaN,México,NaN,4569,[México],"[{'label': 'display', 'lat': 21.15607538245781...",21.156075,-101.709012,NaN,NaN,54791df6498e43be5a5078b1


In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='León Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_soriana.lat, dataframe_filtered_soriana.lng, dataframe_filtered_soriana.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [26]:
search_query_bodega = 'Bodega Aurrera'
radius = 50000
LIMIT = 30

In [27]:
url_bodega = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_bodega, radius, LIMIT)

In [28]:
results_bodega = requests.get(url_bodega).json()

In [29]:
venues_bodega = results_bodega['response']['venues']
# tranform venues into a dataframe
dataframe_bodega = json_normalize(venues_bodega)
dataframe_bodega.head(30)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,561e8648498e59a7c2b020be,Adolfo López Mateos,MX,León,México,Pradera,3872,"[Adolfo López Mateos (Pradera), 37510 León, Gu...","[{'label': 'display', 'lat': 21.11169232887497...",21.111692,-101.647059,37510,Guanajuato,Bodega Aurrera López Mateos,v-1605662218
1,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4f55646ee4b0861b32496aef,Brisas del Lago,MX,León,México,NaN,5817,"[Brisas del Lago, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.16635446530878...",21.166354,-101.649881,NaN,Guanajuato,Bodega Aurrera,v-1605662218
2,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,5e07f91f9939a9000783f246,NaN,MX,León,México,NaN,1240,"[37000 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.113404, 'lng':...",21.113404,-101.686930,37000,Guanajuato,Bodega Aurrera,v-1605662218
3,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4f25be19e4b0b653a4ea7ea1,Camino a Los Naranjos,MX,León,México,Colina Ágata,6790,"[Camino a Los Naranjos (Colina Ágata), León, G...","[{'label': 'display', 'lat': 21.16248103004282...",21.162481,-101.631702,NaN,Guanajuato,Bodega aurrera express,v-1605662218
4,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4d10ecd1b765224b2ad1ed32,NaN,MX,NaN,México,NaN,2576,[México],"[{'label': 'display', 'lat': 21.11991661251871...",21.119917,-101.706599,NaN,NaN,Bodega Aurrera,v-1605662218
5,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",False,4d56b9c9b4deb1f7336b147a,NaN,MX,NaN,México,NaN,2612,[México],"[{'label': 'display', 'lat': 21.13220010755642...",21.132200,-101.658686,NaN,NaN,Bodega Aurrera Obelisco,v-1605662218
6,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,53850f0c498edfec609f05f7,Blvd Francisco Villa Sur #1206,MX,León,México,Prol. Rio Mayo,3986,[Blvd Francisco Villa Sur #1206 (Prol. Rio May...,"[{'label': 'display', 'lat': 21.09169229788461...",21.091692,-101.664664,37557,Guanajuato,Bodega Aurrera Express,v-1605662218
7,"[{'id': '52f2ab2ebcbc57f1066b8b42', 'name': 'B...",False,5399da41498ef219412e5a66,NaN,MX,NaN,México,NaN,2009,[México],"[{'label': 'display', 'lat': 21.13303078052559...",21.133031,-101.698604,NaN,NaN,Bodega Aurrera Espíritu Santo,v-1605662218
8,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,514912d4e4b0631e055c181d,Blvd. Vicente Valtierra esq. Marro,MX,León,México,NaN,2015,"[Blvd. Vicente Valtierra esq. Marro, Leon, Gua...","[{'label': 'display', 'lat': 21.13142761577083...",21.131428,-101.664597,NaN,Guanajuato,Bodega aurrera express,v-1605662218
9,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4e38495d52b1aec580025d87,NaN,MX,Lagos de Moreno,México,NaN,36264,"[Lagos de Moreno, Jalisco, México]","[{'label': 'display', 'lat': 21.35900185420913...",21.359002,-101.923705,NaN,Jalisco,Bodega Aurrera,v-1605662218


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_bodega = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_bodega = dataframe_bodega.loc[:, filtered_columns_bodega]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_bodega['categories'] = dataframe_filtered_bodega.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_bodega.columns = [column.split('.')[-1] for column in dataframe_filtered_bodega.columns]

dataframe_filtered_bodega

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bodega Aurrera López Mateos,Convenience Store,Adolfo López Mateos,MX,León,México,Pradera,3872,"[Adolfo López Mateos (Pradera), 37510 León, Gu...","[{'label': 'display', 'lat': 21.11169232887497...",21.111692,-101.647059,37510,Guanajuato,561e8648498e59a7c2b020be
1,Bodega Aurrera,Convenience Store,Brisas del Lago,MX,León,México,NaN,5817,"[Brisas del Lago, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.16635446530878...",21.166354,-101.649881,NaN,Guanajuato,4f55646ee4b0861b32496aef
2,Bodega Aurrera,Convenience Store,NaN,MX,León,México,NaN,1240,"[37000 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.113404, 'lng':...",21.113404,-101.686930,37000,Guanajuato,5e07f91f9939a9000783f246
3,Bodega aurrera express,Convenience Store,Camino a Los Naranjos,MX,León,México,Colina Ágata,6790,"[Camino a Los Naranjos (Colina Ágata), León, G...","[{'label': 'display', 'lat': 21.16248103004282...",21.162481,-101.631702,NaN,Guanajuato,4f25be19e4b0b653a4ea7ea1
4,Bodega Aurrera,Supermarket,NaN,MX,NaN,México,NaN,2576,[México],"[{'label': 'display', 'lat': 21.11991661251871...",21.119917,-101.706599,NaN,NaN,4d10ecd1b765224b2ad1ed32
5,Bodega Aurrera Obelisco,Miscellaneous Shop,NaN,MX,NaN,México,NaN,2612,[México],"[{'label': 'display', 'lat': 21.13220010755642...",21.132200,-101.658686,NaN,NaN,4d56b9c9b4deb1f7336b147a
6,Bodega Aurrera Express,Convenience Store,Blvd Francisco Villa Sur #1206,MX,León,México,Prol. Rio Mayo,3986,[Blvd Francisco Villa Sur #1206 (Prol. Rio May...,"[{'label': 'display', 'lat': 21.09169229788461...",21.091692,-101.664664,37557,Guanajuato,53850f0c498edfec609f05f7
7,Bodega Aurrera Espíritu Santo,Big Box Store,NaN,MX,NaN,México,NaN,2009,[México],"[{'label': 'display', 'lat': 21.13303078052559...",21.133031,-101.698604,NaN,NaN,5399da41498ef219412e5a66
8,Bodega aurrera express,Convenience Store,Blvd. Vicente Valtierra esq. Marro,MX,León,México,NaN,2015,"[Blvd. Vicente Valtierra esq. Marro, Leon, Gua...","[{'label': 'display', 'lat': 21.13142761577083...",21.131428,-101.664597,NaN,Guanajuato,514912d4e4b0631e055c181d
9,Bodega Aurrera,Supermarket,NaN,MX,Lagos de Moreno,México,NaN,36264,"[Lagos de Moreno, Jalisco, México]","[{'label': 'display', 'lat': 21.35900185420913...",21.359002,-101.923705,NaN,Jalisco,4e38495d52b1aec580025d87


In [31]:
dataframe_filtered_bodega['categories'] = dataframe_filtered_bodega['categories'].astype(str)

In [32]:
possible_bodega = ['Supermarket', 'Market', 'Grocery Store', 'Convenience Store']

In [33]:
dataframe_filtered_bodega = dataframe_filtered_bodega[dataframe_filtered_bodega.categories.isin(possible_bodega)]

In [34]:
dataframe_filtered_bodega

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bodega Aurrera López Mateos,Convenience Store,Adolfo López Mateos,MX,León,México,Pradera,3872,"[Adolfo López Mateos (Pradera), 37510 León, Gu...","[{'label': 'display', 'lat': 21.11169232887497...",21.111692,-101.647059,37510,Guanajuato,561e8648498e59a7c2b020be
1,Bodega Aurrera,Convenience Store,Brisas del Lago,MX,León,México,NaN,5817,"[Brisas del Lago, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.16635446530878...",21.166354,-101.649881,NaN,Guanajuato,4f55646ee4b0861b32496aef
2,Bodega Aurrera,Convenience Store,NaN,MX,León,México,NaN,1240,"[37000 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.113404, 'lng':...",21.113404,-101.686930,37000,Guanajuato,5e07f91f9939a9000783f246
3,Bodega aurrera express,Convenience Store,Camino a Los Naranjos,MX,León,México,Colina Ágata,6790,"[Camino a Los Naranjos (Colina Ágata), León, G...","[{'label': 'display', 'lat': 21.16248103004282...",21.162481,-101.631702,NaN,Guanajuato,4f25be19e4b0b653a4ea7ea1
4,Bodega Aurrera,Supermarket,NaN,MX,NaN,México,NaN,2576,[México],"[{'label': 'display', 'lat': 21.11991661251871...",21.119917,-101.706599,NaN,NaN,4d10ecd1b765224b2ad1ed32
6,Bodega Aurrera Express,Convenience Store,Blvd Francisco Villa Sur #1206,MX,León,México,Prol. Rio Mayo,3986,[Blvd Francisco Villa Sur #1206 (Prol. Rio May...,"[{'label': 'display', 'lat': 21.09169229788461...",21.091692,-101.664664,37557,Guanajuato,53850f0c498edfec609f05f7
8,Bodega aurrera express,Convenience Store,Blvd. Vicente Valtierra esq. Marro,MX,León,México,NaN,2015,"[Blvd. Vicente Valtierra esq. Marro, Leon, Gua...","[{'label': 'display', 'lat': 21.13142761577083...",21.131428,-101.664597,NaN,Guanajuato,514912d4e4b0631e055c181d
9,Bodega Aurrera,Supermarket,NaN,MX,Lagos de Moreno,México,NaN,36264,"[Lagos de Moreno, Jalisco, México]","[{'label': 'display', 'lat': 21.35900185420913...",21.359002,-101.923705,NaN,Jalisco,4e38495d52b1aec580025d87
10,Bodega Aurrera,Supermarket,Luis H. Ducoing 5,MX,Silao,México,NaN,32960,"[Luis H. Ducoing 5, Silao, Guanajuato, México]","[{'label': 'display', 'lat': 20.95115170230896...",20.951152,-101.424225,NaN,Guanajuato,4df79129c65b87473b079d38
14,Bodega Aurrera,Grocery Store,Blvd Hilario Medina 902,MX,León,México,Vicente Valtierra,3601,"[Blvd Hilario Medina 902 (Vicente Valtierra), ...","[{'label': 'display', 'lat': 21.14796208374651...",21.147962,-101.659273,NaN,Guanajuato,4c0d0a402466a59308ec7621


In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='León Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_bodega.lat, dataframe_filtered_bodega.lng, dataframe_filtered_bodega.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [39]:
df_final = pd.concat([dataframe_filtered, dataframe_filtered_soriana, dataframe_filtered_bodega])

In [40]:
df_final

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Walmart,Big Box Store,Blvd. Manuel J. Clouthier 302,MX,León,México,Bosques del Campestre,4320,[Blvd. Manuel J. Clouthier 302 (Bosques del Ca...,"[{'label': 'display', 'lat': 21.16149885883102...",21.161499,-101.691081,37125,Guanajuato,4bbe2a449474c9b6712ed9b6
1,Walmart,Big Box Store,Blvd. Juan José Torres Landa 1601,MX,León,México,NaN,3601,"[Blvd. Juan José Torres Landa 1601, 37178 León...","[{'label': 'display', 'lat': 21.09832200719789...",21.098322,-101.660449,37178,Guanajuato,4be5d857d4f7c9b648052620
2,Walmart,Big Box Store,Galerias Las Torres,MX,León,México,Blvd. Juan Alonso de Torres 1325,4503,[Galerias Las Torres (Blvd. Juan Alonso de Tor...,"[{'label': 'display', 'lat': 21.14700345173944...",21.147003,-101.646720,37178,Guanajuato,4c09c20a340720a1b4708593
3,Walmart,Big Box Store,Blvd. La Luz,MX,León,México,Blvd. Delta,7653,"[Blvd. La Luz (Blvd. Delta), 3720 León, Guanaj...","[{'label': 'display', 'lat': 21.11007609214700...",21.110076,-101.609840,3720,Guanajuato,50358829e4b0aab52bd7e343
4,Walmart Via Alta,Big Box Store,NaN,MX,León,México,NaN,9476,"[37295 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.079158, 'lng':...",21.079158,-101.604270,37295,Guanajuato,5dbb9a306dfe8f00086dd670
5,Walmart La Antorcha,Supermarket,Blvd. Morelos,MX,León,México,NaN,7576,"[Blvd. Morelos, 37665 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.17264957202568...",21.172650,-101.732696,37665,Guanajuato,54a181e8498e3242ba86a427
6,Walmart,Big Box Store,Blvd. Francisco Villa N° 1402,MX,León,México,NaN,3132,"[Blvd. Francisco Villa N° 1402, 37278 León, Gu...","[{'label': 'display', 'lat': 21.12962264302637...",21.129623,-101.652633,37278,Guanajuato,54a186dd498e8fb585d6732f
8,Walmart Francisco Villa,Big Box Store,Boulevard Francisco Villa 1402,MX,Villa de laa flores,México,Fray Daniel Mireles,3578,[Boulevard Francisco Villa 1402 (Fray Daniel M...,"[{'label': 'display', 'lat': 21.12338974610575...",21.123390,-101.647636,NaN,Guanajuato,5494e47a498ecffa71a10e3e
9,Walmart Punta Este,Supermarket,NaN,MX,León,México,NaN,7816,"[37179 León, Guanajuato, México]","[{'label': 'display', 'lat': 21.132607, 'lng':...",21.132607,-101.607440,37179,Guanajuato,5bf1d8c182a750002c071844
0,Soriana,Supermarket,Malecon,MX,León,México,NaN,2893,"[Malecon, León, Guanajuato, México]","[{'label': 'display', 'lat': 21.1067714166875,...",21.106771,-101.660859,NaN,Guanajuato,4c17acaec2dfc9283f61a869


In [41]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='León Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_final.lat, df_final.lng, df_final.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map